<a href="https://colab.research.google.com/github/Worachet-Ch/project2022/blob/main/random_nan_and_imputation_with_weight_variable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd #นำเข้า pandas แทนที่ด้วย pd
import os# os ใช้จัดการกับไฟล์ ว่าอยู่ที่ไหน
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as mno
import missingno as msno
from sklearn import linear_model
%matplotlib inline
import matplotlib
import sklearn 
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
import time

In [2]:
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/project' #เข้า drive
data_file_path = os.path.join(path,'data_notmissing.csv') #ไปที่ไฟล์ confirmed-cases-since-280265.csv
data = pd.read_csv(data_file_path) #อ่านไฟล์ csv

Mounted at /content/drive


In [3]:
data.drop({'Unnamed: 0','telomere length (kb) '},1,inplace = True)

<ipython-input-3-4cc622f169ae>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data.drop({'Unnamed: 0','telomere length (kb) '},1,inplace = True)


In [58]:
data

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [ ]:
t_start_t = time.time()

# 10 % random nan

In [22]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'weight{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*10/100)) 
  index_nan = np.random.choice(globals()[f'weight{i}']['Weight'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'weight{i}']['Weight'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.2954585552215576 วินาที
เวลาที่ใช้ในการประมวลผล 0.004924309253692627 นาที


In [23]:
weight1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation using Mean

In [24]:
from sklearn.impute import SimpleImputer

In [25]:
t_start = time.time()
mean10_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean10_{i}'] = mean10_imputer.fit_transform(globals()[f'weight{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN10_{i}'] = mean_squared_error(data, globals()[f'imputed_mean10_{i}']) # for loop mean_squared_error
t_end = time.time()
t10_MEAN = (t_end- t_start)

In [26]:
t_start = time.time()
mse_mean10_total = 0 
MSE_MEAN10_1001 = 0
for i in range(0,1001):
  mse_mean10_total = mse_mean10_total + globals()[f'MSE_MEAN10_{i+1}']
AMSE_weight10_mean = mse_mean10_total/1000
print(f'AMSE_weight10_mean = {AMSE_weight10_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight10_mean = 0.2982275919348313
เวลาที่ใช้ในการประมวลผล 0.003585338592529297 วินาที
เวลาที่ใช้ในการประมวลผล 5.975564320882161e-05 นาที


## Imputation using the miss forest

In [27]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49 kB 4.2 MB/s 


In [ ]:
t_start = time.time()
MF10_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf10_{i}'] = MF10_imputer.fit_transform(globals()[f'weight{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF10_{i}'] = mean_squared_error(data, globals()[f'imputed_mf10_{i}']) # for loop mean_squared_error
t_end = time.time()
t10_MF= (t_end- t_start)

In [29]:
t_start = time.time()
mse_mf10_total = 0 
MSE_MF10_1001 = 0
for i in range(0,1001):
  mse_mf10_total = mse_mf10_total + globals()[f'MSE_MF10_{i+1}']
AMSE_weight10_mf = mse_mf10_total/1000
print(f'AMSE_weight10_mf = {AMSE_weight10_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight10_mf = 0.03013601033056814
เวลาที่ใช้ในการประมวลผล 0.0038518905639648438 วินาที
เวลาที่ใช้ในการประมวลผล 6.419817606608073e-05 นาที


## Imputation using the KNN

In [30]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [107]:
t_start = time.time()
KNN10_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN10_{i}'] = KNN10_imputer.fit_transform(globals()[f'weight{i}'].copy())
  globals()[f'MSE_KNN10_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN10_{i}'])
t_end = time.time()
t10_KNN= (t_end- t_start)

In [108]:
t_start = time.time()
mse_knn10_total = 0 
MSE_KNN10_1001 = 0
for i in range(0,1001):
  mse_knn10_total = mse_knn10_total + globals()[f'MSE_KNN10_{i+1}']
AMSE_weight10_knn = mse_knn10_total/1000
print(f'AMSE_weight10_knn = {AMSE_weight10_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight10_knn = 0.1805482678733033
เวลาที่ใช้ในการประมวลผล 0.0017578601837158203 วินาที
เวลาที่ใช้ในการประมวลผล 2.9297669728597007e-05 นาที


## Imputation using the MICE

In [33]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr10_{i}'] = imp.fit_transform(globals()[f'weight{i}'].copy())
  globals()[f'MSE_lr10_{i}'] = mean_squared_error(data, globals()[f'imputed_lr10_{i}'])
t_end = time.time()
t10_LR = (t_end- t_start)

In [35]:
t_start = time.time()
mse_lr10_total = 0 
MSE_lr10_1001 = 0
for i in range(0,1001):
  mse_lr10_total = mse_lr10_total + globals()[f'MSE_lr10_{i+1}']
AMSE_weight10_lr = mse_lr10_total/1000
print(f'AMSE_weight10_lr = {AMSE_weight10_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight10_lr = 0.004184412666016818
เวลาที่ใช้ในการประมวลผล 0.003085613250732422 วินาที
เวลาที่ใช้ในการประมวลผล 5.142688751220703e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm10_{i}'] = svm_svr.fit_transform(globals()[f'weight{i}'].copy())
  globals()[f'MSE_svm10_{i}'] = mean_squared_error(data, globals()[f'imputed_svm10_{i}'])
t_end = time.time()
t10_SVM = (t_end- t_start)

In [37]:
t_start = time.time()
mse_svm10_total = 0 
MSE_svm10_1001 = 0
for i in range(0,1001):
  mse_svm10_total = mse_svm10_total + globals()[f'MSE_svm10_{i+1}']
AMSE_weight10_svm = mse_svm10_total/1000
print(f'AMSE_weight10_svm = {AMSE_weight10_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight10_svm = 0.24436632066858974
เวลาที่ใช้ในการประมวลผล 0.006444692611694336 วินาที
เวลาที่ใช้ในการประมวลผล 0.00010741154352823893 นาที


## Imputation using the EM

### impy.em

In [ ]:
!pip install impyute
import impyute as impy

In [55]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy10_{i}'] = impy.em(globals()[f'weight{i}'].values.copy())
  globals()[f'MSE_impy10_{i}'] = mean_squared_error(data, globals()[f'imputed_impy10_{i}'])
t_end = time.time()
t10_EM = (t_end- t_start)

SyntaxError: ignored

In [40]:
t_start = time.time()
mse_impy10_total = 0 
MSE_impy10_1001 = 0
for i in range(0,1001):
  mse_impy10_total = mse_impy10_total + globals()[f'MSE_impy10_{i+1}']
AMSE_weight10_impy = mse_impy10_total/1000
print(f'AMSE_weight10_impy = {AMSE_weight10_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight10_impy = 0.5530573965089465
เวลาที่ใช้ในการประมวลผล 0.010115623474121094 วินาที
เวลาที่ใช้ในการประมวลผล 0.0001685937245686849 นาที


# 20 % random nan

In [69]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'weight20_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*20/100)) 
  index_nan = np.random.choice(globals()[f'weight20_{i}']['Weight'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'weight20_{i}']['Weight'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.2006697654724121 วินาที
เวลาที่ใช้ในการประมวลผล 0.0033444960912068686 นาที


In [70]:
weight20_1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation using Mean

In [71]:
from sklearn.impute import SimpleImputer

In [72]:
t_start = time.time()
mean20_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean20_{i}'] = mean10_imputer.fit_transform(globals()[f'weight20_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN20_{i}'] = mean_squared_error(data, globals()[f'imputed_mean20_{i}']) # for loop mean_squared_error
t_end = time.time()
t20_MEAN = (t_end- t_start)

In [73]:
t_start = time.time()
mse_mean20_total = 0 
MSE_MEAN20_1001 = 0
for i in range(0,1001):
  mse_mean20_total = mse_mean20_total + globals()[f'MSE_MEAN20_{i+1}']
AMSE_weight20_mean = mse_mean20_total/1000
print(f'AMSE_weight20_mean = {AMSE_weight20_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight20_mean = 0.5866671506272794
เวลาที่ใช้ในการประมวลผล 0.001196146011352539 วินาที
เวลาที่ใช้ในการประมวลผล 1.993576685587565e-05 นาที


## Imputation using the miss forest

In [74]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF20_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf20_{i}'] = MF20_imputer.fit_transform(globals()[f'weight20_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF20_{i}'] = mean_squared_error(data, globals()[f'imputed_mf20_{i}']) # for loop mean_squared_error
t_end = time.time()
t20_MF = (t_end- t_start)

In [76]:
t_start = time.time()
mse_mf20_total = 0 
MSE_MF20_1001 = 0
for i in range(0,1001):
  mse_mf20_total = mse_mf20_total + globals()[f'MSE_MF20_{i+1}']
AMSE_weight20_mf = mse_mf20_total/1000
print(f'AMSE_weight20_mf = {AMSE_weight20_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight20_mf = 0.06149018353770736
เวลาที่ใช้ในการประมวลผล 0.002244710922241211 วินาที
เวลาที่ใช้ในการประมวลผล 3.741184870402018e-05 นาที


## Imputation using the KNN

In [77]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [109]:
t_start = time.time()
KNN20_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN20_{i}'] = KNN10_imputer.fit_transform(globals()[f'weight20_{i}'].copy())
  globals()[f'MSE_KNN20_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN20_{i}'])
t_end = time.time()
t20_KNN = (t_end- t_start)

In [110]:
t_start = time.time()
mse_knn20_total = 0 
MSE_KNN20_1001 = 0
for i in range(0,1001):
  mse_knn20_total = mse_knn20_total + globals()[f'MSE_KNN20_{i+1}']
AMSE_weight20_knn = mse_knn20_total/1000
print(f'AMSE_weight20_knn = {AMSE_weight20_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight20_knn = 0.3633119561700464
เวลาที่ใช้ในการประมวลผล 0.005179882049560547 วินาที
เวลาที่ใช้ในการประมวลผล 8.633136749267579e-05 นาที


## Imputation using the MICE

In [80]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp20 = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr20_{i}'] = imp20.fit_transform(globals()[f'weight20_{i}'].copy())
  globals()[f'MSE_lr20_{i}'] = mean_squared_error(data, globals()[f'imputed_lr20_{i}'])
t_end = time.time()
t20_LR = (t_end- t_start)

In [82]:
t_start = time.time()
mse_lr20_total = 0 
MSE_lr20_1001 = 0
for i in range(0,1001):
  mse_lr20_total = mse_lr20_total + globals()[f'MSE_lr20_{i+1}']
AMSE_weight20_lr = mse_lr20_total/1000
print(f'AMSE_weight20_lr = {AMSE_weight20_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight20_lr = 0.011561803266590068
เวลาที่ใช้ในการประมวลผล 0.003604888916015625 วินาที
เวลาที่ใช้ในการประมวลผล 6.008148193359375e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm20_{i}'] = svm_svr.fit_transform(globals()[f'weight20_{i}'].copy())
  globals()[f'MSE_svm20_{i}'] = mean_squared_error(data, globals()[f'imputed_svm20_{i}'])
t_end = time.time()
t20_SVM = (t_end- t_start)

In [84]:
t_start = time.time()
mse_svm20_total = 0 
MSE_svm20_1001 = 0
for i in range(0,1001):
  mse_svm20_total = mse_svm20_total + globals()[f'MSE_svm20_{i+1}']
AMSE_weight20_svm = mse_svm20_total/1000
print(f'AMSE_weight10_svm = {AMSE_weight20_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight10_svm = 0.49041936875791103
เวลาที่ใช้ในการประมวลผล 0.0034570693969726562 วินาที
เวลาที่ใช้ในการประมวลผล 5.76178232828776e-05 นาที


## Imputation using the EM

### impy.em

In [85]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [86]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy20_{i}'] = impy.em(globals()[f'weight20_{i}'].values.copy())
  globals()[f'MSE_impy20_{i}'] = mean_squared_error(data, globals()[f'imputed_impy20_{i}'])
t_end = time.time()
t20_EM = (t_end- t_start)

In [87]:
t_start = time.time()
mse_impy20_total = 0 
MSE_impy20_1001 = 0
for i in range(0,1001):
  mse_impy20_total = mse_impy20_total + globals()[f'MSE_impy20_{i+1}']
AMSE_weight20_impy = mse_impy20_total/1000
print(f'AMSE_weight20_impy = {AMSE_weight20_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight20_impy = 1.0935381960555157
เวลาที่ใช้ในการประมวลผล 0.004761219024658203 วินาที
เวลาที่ใช้ในการประมวลผล 7.935365041097005e-05 นาที


# 30 % random nan

In [88]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'weight30_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*30/100)) 
  index_nan = np.random.choice(globals()[f'weight30_{i}']['Weight'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'weight30_{i}']['Weight'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.25695323944091797 วินาที
เวลาที่ใช้ในการประมวลผล 0.004282553990681966 นาที


In [89]:
weight30_1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation using Mean

In [90]:
from sklearn.impute import SimpleImputer

In [91]:
t_start = time.time()
mean30_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean30_{i}'] = mean30_imputer.fit_transform(globals()[f'weight30_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN30_{i}'] = mean_squared_error(data, globals()[f'imputed_mean30_{i}']) # for loop mean_squared_error
t_end = time.time()
t30_MEAN = (t_end- t_start)

In [92]:
t_start = time.time()
mse_mean30_total = 0 
MSE_MEAN30_1001 = 0
for i in range(0,1001):
  mse_mean30_total = mse_mean30_total + globals()[f'MSE_MEAN30_{i+1}']
AMSE_weight30_mean = mse_mean30_total/1000
print(f'AMSE_weight30_mean = {AMSE_weight30_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight30_mean = 0.8847872255080415
เวลาที่ใช้ในการประมวลผล 0.011403560638427734 วินาที
เวลาที่ใช้ในการประมวลผล 0.00019005934397379556 นาที


## Imputation using the miss forest

In [93]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF30_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf30_{i}'] = MF30_imputer.fit_transform(globals()[f'weight30_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF30_{i}'] = mean_squared_error(data, globals()[f'imputed_mf30_{i}']) # for loop mean_squared_error
t_end = time.time()
t30_MF = (t_end- t_start)

In [95]:
t_start = time.time()
mse_mf30_total = 0 
MSE_MF30_1001 = 0
for i in range(0,1001):
  mse_mf30_total = mse_mf30_total + globals()[f'MSE_MF30_{i+1}']
AMSE_weight30_mf = mse_mf30_total/1000
print(f'AMSE_weight30_mf = {AMSE_weight30_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight30_mf = 0.09729602599371548
เวลาที่ใช้ในการประมวลผล 0.004587888717651367 วินาที
เวลาที่ใช้ในการประมวลผล 7.646481196085611e-05 นาที


## Imputation using the KNN

In [96]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [111]:
t_start = time.time()
KNN30_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN30_{i}'] = KNN30_imputer.fit_transform(globals()[f'weight30_{i}'].copy())
  globals()[f'MSE_KNN30_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN30_{i}'])
t_end = time.time()
t30_KNN = (t_end- t_start)

In [112]:
t_start = time.time()
mse_knn30_total = 0 
MSE_KNN30_1001 = 0
for i in range(0,1001):
  mse_knn30_total = mse_knn30_total + globals()[f'MSE_KNN30_{i+1}']
AMSE_weight30_knn = mse_knn30_total/1000
print(f'AMSE_weight20_knn = {AMSE_weight30_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight20_knn = 0.5635913499916201
เวลาที่ใช้ในการประมวลผล 0.004913330078125 วินาที
เวลาที่ใช้ในการประมวลผล 8.188883463541666e-05 นาที


## Imputation using the MICE

In [99]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp20 = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr30_{i}'] = imp20.fit_transform(globals()[f'weight30_{i}'].copy())
  globals()[f'MSE_lr30_{i}'] = mean_squared_error(data, globals()[f'imputed_lr30_{i}'])
t_end = time.time()
t30_LR = (t_end- t_start)

In [101]:
t_start = time.time()
mse_lr30_total = 0 
MSE_lr30_1001 = 0
for i in range(0,1001):
  mse_lr30_total = mse_lr30_total + globals()[f'MSE_lr30_{i+1}']
AMSE_weight30_lr = mse_lr30_total/1000
print(f'AMSE_weight30_lr = {AMSE_weight30_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight30_lr = 0.02564107138342625
เวลาที่ใช้ในการประมวลผล 0.005902528762817383 วินาที
เวลาที่ใช้ในการประมวลผล 9.837547938028972e-05 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm30_{i}'] = svm_svr.fit_transform(globals()[f'weight30_{i}'].copy())
  globals()[f'MSE_svm30_{i}'] = mean_squared_error(data, globals()[f'imputed_svm30_{i}'])
t_end = time.time()
t30_SVM = (t_end- t_start)

In [103]:
t_start = time.time()
mse_svm30_total = 0 
MSE_svm30_1001 = 0
for i in range(0,1001):
  mse_svm30_total = mse_svm30_total + globals()[f'MSE_svm30_{i+1}']
AMSE_weight30_svm = mse_svm30_total/1000
print(f'AMSE_weight30_svm = {AMSE_weight30_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight30_svm = 0.7556074545181287
เวลาที่ใช้ในการประมวลผล 0.003670215606689453 วินาที
เวลาที่ใช้ในการประมวลผล 6.117026011149088e-05 นาที


## Imputation using the EM

### impy.em

In [ ]:
!pip install impyute
import impyute as impy

In [105]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy30_{i}'] = impy.em(globals()[f'weight30_{i}'].values.copy())
  globals()[f'MSE_impy30_{i}'] = mean_squared_error(data, globals()[f'imputed_impy30_{i}'])
t_end = time.time()
t30_EM = (t_end- t_start)

In [106]:
t_start = time.time()
mse_impy30_total = 0 
MSE_impy30_1001 = 0
for i in range(0,1001):
  mse_impy30_total = mse_impy30_total + globals()[f'MSE_impy30_{i+1}']
AMSE_weight30_impy = mse_impy30_total/1000
print(f'AMSE_weight30_impy = {AMSE_weight30_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_weight30_impy = 1.6645229318389554
เวลาที่ใช้ในการประมวลผล 0.0030002593994140625 วินาที
เวลาที่ใช้ในการประมวลผล 5.0004323323567705e-05 นาที


# เวลาทั้งหมดในการรัน

## second time

In [142]:
data_t_sec = {'Method':['MEAN','Miss Forest','KNN','MICE-LR','MICE-SVM','EM'],'10%':[t10_MEAN,t10_MF,t10_KNN,t10_LR,t10_SVM,t10_EM],
      '20%':[t20_MEAN,t20_MF,t20_KNN,t20_LR,t20_SVM,t20_EM],'30%':[t30_MEAN,t30_MF,t30_KNN,t30_LR,t30_SVM,t30_EM]}

In [143]:
data_second = pd.DataFrame.from_dict(data_t_sec)
data_second

,Method,10%,20%,30%
0,MEAN,4.844099,4.838218,5.666173
1,Miss Forest,1180.364142,1152.712039,1043.665598
2,KNN,9.264600,7.478876,14.855747
3,MICE-LR,68.927913,69.390142,69.912066
4,MICE-SVM,146.346689,150.789544,150.705075
5,EM,11.145165,24.053548,29.484782
